In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
# Let's load the data set.
# 'with_info=True' to get information about version, features and number of instances.
# 'as_supervised=True' to load the dataset in a 2-tuple structure (input, target) 
# 'as_supervised=False' would return a dictionary

mnist_dataset, mnist_info = tfds.load(name="mnist", with_info= True, as_supervised= True)

In [ ]:
mnist_info

In [ ]:
# Training and testing datasets can be easily extracted.

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# I will use 10% of our training dataset as validation.
# 'tf.cast' casts a tensor to a new type.

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1 * mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


# Since the possible values for the inputs are between 0 and 255, 
# we can scale them between 0 and 1 by dividing each element by 255.

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
# 'map' allows us to apply a custom transformation to a given dataset

scaled_train_validation_data = mnist_train.map(scale)

# Let's scale the test data to have the same size as train and validation.
# No need to shuffle as we won't be training with it
# A single batch equal to the size of the test data will suffice.

test_data = mnist_test.map(scale)

In [ ]:
# I will define a buffer size to save memory, this is mostly useful while working big data.
# with this TF only stores BUFFER_SIZE samples in memory at a time and shuffles them.
# If BUFFER_SIZE=1, there will actually be no shuffling, and of course if BUFFER_SIZE >= number of samples, mixing would be uniform.
buf_size = 10000

# I will easily use the shuffle method.
shuffled_train_validation_data = scaled_train_validation_data.shuffle(buf_size)

# I create a batch with a batch size equal to the total number of validation samples
# And the rest would be the train data.

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

# Let's determine a batch size.
batch_size = 50

# batching the train data would be very helpful when we train, as we would be able to iterate over the different batches.
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# 'as_supervized=True' takes the next batch as we have a 2-tuple structure.
validation_inputs, validation_targets = next(iter(validation_data))

In [ ]:
input_size = 784
output_size = 10
# I will use same hidden layer size for all hidden layers
hidden_layer_size = 150

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation="tanh"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="tanh"),
                             tf.keras.layers.Dense(output_size, activation="softmax")
                             ])

In [ ]:
# I will define the optimizer, the loss function, and the metrics to get at each iteration.

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# The maximum number of epochs
max_epochs = 10

# Let's set up an early stop mechanism with patience=2 so our model will be somewhat tolerant of random validation loss increase
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# The accuracy shows in what % of the cases the outputs were equal to the targets.
# The val_accuracy shows the true accuracy of the model.
model.fit(train_data, epochs= max_epochs, callbacks=[early_stopping], validation_data=(validation_inputs,validation_targets), verbose=2 )



I hope that you find it useful. As I will be happy to improve myself, your comments and feedbacks are always welcome, as are suggestions for additional information that could usefully be included. Thank you! 🌸😊